In [10]:
import sys
import os
import shutil
from scipy.io.wavfile import read, write
from pydub import AudioSegment
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
os.chdir("/Users/venkatakrishnanvk/Desktop/Music Technology/Datasets/carnatic")

In [4]:
dest_folder = "/Users/venkatakrishnanvk/Desktop/Music Technology/Datasets/carnatic/violin_solo_dataset/vocal/"

# For copying of all multirack vocals

In [4]:
for path in os.scandir(dest_folder):
    if os.path.isfile(path.path):
        os.remove(path.path)

In [52]:
for path, dirs, files in os.walk(os.getcwd()):
    for file in files:
        if file.endswith("multitrack-vocal.mp3"):
            src = path+"/"+file
            if os.path.exists(dest_folder+file):
                continue
            shutil.copy(src, dest_folder+file)

# Chopping to 10 minute audios

In [53]:
for path in os.scandir(dest_folder):
    if os.path.isfile(path.path):
        print(path.name[:-4])

Tulasi Bilva.multitrack-vocal
Shloka Sri Ramachandra Shrita Parijata.multitrack-vocal
Sri Guru Na Palitosmi.multitrack-vocal
Siva Siva Siva Yenarada.multitrack-vocal
Chandrashekharam Sada Bhajeham.multitrack-vocal
Prathi Vaaram Vaaram.multitrack-vocal
Shankari Shankuru.multitrack-vocal
Eppadi Padinaro.multitrack-vocal
Chenduril Nindradum Kanda.multitrack-vocal
Thamarai Kangal.multitrack-vocal
Lokavana Chatura.multitrack-vocal
Sarasuda Ninne Kori.multitrack-vocal
Thillana Vasanta.multitrack-vocal
Velum Mayilume.multitrack-vocal
Parvathi Ninnu Neranammiti.multitrack-vocal
Etulabrotuva.multitrack-vocal
Jaya Jaya Padmanabhanujese.multitrack-vocal
Devadi Devanukku Jaya Mangalam.multitrack-vocal
Kannanai Paadu Maname.multitrack-vocal
Ramabhi Rama Manasu.multitrack-vocal
Vinakayunna Della.multitrack-vocal
Tillana.multitrack-vocal
Shri Guruguhasya.multitrack-vocal
Thillana Hameerkalyani.multitrack-vocal
Krishna Nee Begane Baaro.multitrack-vocal
Hiranmayeem.multitrack-vocal
Brochevarevarura.mul

In [54]:
for path in os.scandir(dest_folder):
    if os.path.isfile(path.path):
        # fs, audio = read(path.path)
        audio_data = AudioSegment.from_file(path.path, "mp3")
        fs = int(audio_data.frame_rate)
        if len(audio_data)/60000 > 10: # 10 minute chunks
            pointer = 0
            while True:
                new_audio = audio_data[pointer*600000:(pointer+1)*600000]
                new_audio.export(dest_folder+path.name[:-4]+str(pointer+1)+".mp3", format = "mp3")
                pointer+=1
                if len(new_audio) != 600000:
                    os.remove(path.path)
                    break

In [36]:
os.chdir(dest_folder)

# Splitting source separated audio into 5 second chunks

In [5]:
directory = "/Users/venkatakrishnanvk/Desktop/Music Technology/Datasets/carnatic/violin_solo_dataset/vocal/vocalists"

In [6]:
count = len(os.listdir(directory)) + 1
count

5460

In [15]:
count = 581

In [16]:
def pydub_to_np(audio: AudioSegment) -> (np.ndarray, int):
    """
    Converts pydub audio segment into np.float32 of shape [duration_in_seconds*sample_rate, channels],
    where each value is in range [-1.0, 1.0]. 
    Returns tuple (audio_np_array, sample_rate).
    """
    return np.array(audio.get_array_of_samples(), dtype=np.float32).reshape((-1, audio.channels)) / (
            1 << (8 * audio.sample_width - 1)), audio.frame_rate


In [17]:
for path, dirs, files in os.walk(dest_folder+"ss_data"):
    for file in files:
        if file == "vocals.mp3":
            audio_data = AudioSegment.from_file(path+"/"+file, "mp3")
            fs = int(audio_data.frame_rate)
            pointer = 0
            while True:
                new_audio = audio_data[pointer*3000:(pointer+1)*3000] # 5 s of audio
                new_audio_np = pydub_to_np(new_audio)[0]
                if not np.isnan(np.mean(np.abs(new_audio_np.T[0]))) and np.mean(np.abs(new_audio_np.T[0])) > 0.0001:
                    pointer+=1
                    write("violin_solo_dataset/vocal/vocalists/vocal"+str(count)+".wav", 44100, new_audio_np.T[0])
                    # new_audio.export("violin_solo_dataset/vocal/vocalists/vocal"+str(count)+".wav", format = "wav")
                    # sys.exit()
                    count+=1
                    pointer+=1
                else:
                    pointer+=1
                if len(new_audio)<3000:
                    break